In [1]:
import torch
import random
import numpy as np
from functools import partial

In [2]:
# pip install torch torch_geometric torch-scatter pandas scikit-learn wandb
# python -m pip install gurobipy

In [3]:
from temp.data.gen import parallel_generate_problem, parallel_generate_solutions
from temp.data.problem import setcover
from temp.data.info import ModelInfo
from temp.data.dataset import ModelGraphDataset
from temp.data.aug import parallel_augment_info, augment_info


from temp.solver.utils import solve_inst


In [4]:
PRE_TRAIN_DIR = "temp/pre_train"
TRAIN_DIR = "temp/train"
VALID_DIR = "temp/valid"

In [5]:
# parallel_generate_problem(setcover, PRE_TRAIN_DIR, 100, 10)
# parallel_generate_solutions(PRE_TRAIN_DIR, 10)

# parallel_generate_problem(setcover, TRAIN_DIR, 10000, 10)
# parallel_generate_solutions(TRAIN_DIR, 10)

# parallel_generate_problem(setcover, VALID_DIR, 100, 10)
# parallel_generate_solutions(VALID_DIR, 10)

In [6]:
m = setcover()
model = m

# model.setParam("OutputFlag", 0)
model.setParam("PoolSolutions", 10)
model.setParam("PoolSearchMode", 2)
model.optimize()

vs = model.getVars()
obj_val_and_sols = []

for i in range(model.SolCount):
    # TODO: setting solution number actually takes long time try to optimize
    # TODO: add function to round by tolerance
    model.params.SolutionNumber = i
    obj_val = model.PoolObjVal
    s = [obj_val] + [v.Xn for v in vs]
    obj_val_and_sols.append(s)

info = ModelInfo.from_model(m)
info.var_info.sols = np.array(obj_val_and_sols)

Restricted license - for non-production use only - expires 2026-11-23
Set parameter PoolSolutions to value 10
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
PoolSearchMode  2

Optimize a model with 128 rows, 256 columns and 3196 nonzeros
Model fingerprint: 0x9c3527fa
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 214.0000000
Presolve time: 0.00s
Presolved: 128 rows, 256 columns, 3196 nonzeros
Variable types: 0 continuous, 256 integer (256 binary)
Found heuristic solution: objective 203.0000000

Root relaxation: objective 7.145291e+01, 206 iterations, 0.00

In [7]:
# from temp.data.dataset import info_to_data
# d = info_to_data(info)
# aug = partial(parallel_augment_info, n=100)

In [8]:
augment_info(info)

[(0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (1.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (1.0, 1.0, 'B'), (0.0, 1.0, 'B'), (1.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (1.0, 1.0, 'B'), (1.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (1.0, 1.0, 'B'), (1.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (1.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (1.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (1.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (1.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B'), (0.0, 1.0, 'B

In [9]:
pre_train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=augment_info)
train_d = ModelGraphDataset(TRAIN_DIR, augment=augment_info)
valid_d = ModelGraphDataset(VALID_DIR)

data = pre_train_d[0][1]
var_feature_size = data.var_node_features.size(-1)
con_feature_size = data.con_node_features.size(-1) 

In [10]:
import pandas as pd
CFG = pd.read_excel("temp/trained_models/setcover_model_configs.xlsx", index_col=0).loc[0].to_dict()
CFG["num_epochs"] = 16
CFG["num_layers"] = 8
CFG["hidden"] = 16

In [11]:
from temp.model.utils import get_model
from temp.model.trainer import train

In [12]:
model_name, model, criterion, optimizer, scheduler = get_model(".", var_feature_size, con_feature_size, n_batches=1, **CFG)

In [13]:
scheduler.total_steps = 10000

In [14]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()
    np.random.seed(worker_seed)
    random.seed(worker_seed)

from torch_geometric.loader import DataLoader
pretrain_loader = DataLoader(pre_train_d, batch_size=8, num_workers=4, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(train_d, batch_size=8, shuffle=True, num_workers=4, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
val_loader = DataLoader(valid_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))

TypeError: DataLoader.__init__() got an unexpected keyword argument 'num_worker'

In [15]:
scheduler.total_steps = 1000000

In [16]:
train(model_name, model, criterion, optimizer, scheduler, pretrain_loader, train_loader, val_loader, CFG, False, "./")

>> Training starts on the current device cpu
>> Pretraining for prenorm...


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]


>> Epoch 1 ----------------------------------------------------------------------------------------------------
Training... 0


 10%|█         | 131/1250 [02:19<21:48,  1.17s/it]

...

val_loss                 0.204250
val_acc                  0.943400
val_f1                   0.508254
val_precision            0.617089
val_recall               0.432053
val_evidence_succ       42.052574
val_evidence_fail        4.703334
val_uncertainty_succ     0.110928
val_uncertainty_fail     0.534775
val_true_bias            0.067644
val_pred_bias            0.047644
val_soft_pred_bias       0.048490
val_bias_error           0.020000
val_lr                   0.000371
dtype: float32
>> Epoch 6 ----------------------------------------------------------------------------------------------------
Training... 6250


  8%|▊         | 94/1250 [00:58<11:49,  1.63it/s]

>>> tensor(1.)


 52%|█████▏    | 644/1250 [06:54<06:29,  1.55it/s]


KeyboardInterrupt: 

In [17]:
from temp.data.dataset import info_to_data

In [54]:
m = setcover(512, 512)

In [55]:
info = ModelInfo.from_model(m)
data = info_to_data(info)

In [32]:
with torch.no_grad():
    logits = model(data)

In [33]:
binary_mask = torch.ones(len(logits), dtype=bool)

In [34]:
from temp.solver import upr

In [35]:
probs, preds = upr.get_predictions(logits, binary_mask)

In [36]:
unc = upr.get_uncertainty(logits)

In [37]:
threshold = upr.get_threshold(unc)

In [38]:
import json
from tempfile import NamedTemporaryFile

import gurobipy as gp


def fix_var(inst, idxs, vals):
    assert len(idxs) == len(vals)
    bounds = {}
    vs = inst.getVars()
    for idx, val in zip(idxs, vals):
        v = vs[idx]
        bounds[idx] = (v.lb, v.ub)
        v.setAttr("lb", val)
        v.setAttr("ub", val)
    inst.update()
    return bounds


def unfix_var(inst, idxs, bounds):
    assert len(idxs) == len(bounds)
    vs = inst.getVars()
    print(idxs, bounds)
    for i, (lb, ub) in zip(idxs, bounds):
        vs[i].setAttr("lb", lb)
        vs[i].setAttr("ub", ub)


def get_iis_vars(inst):
    try:
        inst.computeIIS()
    except Exception as e:
        print(e)
        if "Cannot compute IIS on a feasible model" in str(e):
            return set()
        raise e

    with NamedTemporaryFile(suffix=".ilp", mode="w+") as f:
        inst.write(f.name)
        f.seek(0)
        return set(f.read().split())


def set_starts(inst, starts):
    vs = inst.getVars()
    for i, s in starts.items():
        vs[i].setAttr("lb", s)


def solve_inst(inst):
    vs = inst.getVars()
    inst.optimize()
    return inst.getAttr("X", vs)


def repair(inst, fixed: set, bounds: dict):
    old_iis_method = getattr(inst, "IISMethod", -1)
    inst.setParam("IISMethod", 0)

    vs = inst.getVars()
    ns = inst.getAttr("varName", vs)
    name_to_idx = {n: i for i, n in enumerate(ns)}

    freed = set()
    while iis_var_names := get_iis_vars(inst):
        for n in iis_var_names:
            if n not in name_to_idx:
                continue

            var_idx = name_to_idx[n]
            if var_idx not in fixed:
                continue

            if var_idx in freed:
                continue

            lb, ub = bounds[var_idx]
            vs[var_idx].lb = lb
            vs[var_idx].ub = ub
            freed.add(var_idx)

    inst.setParam("IISMethod", old_iis_method)
    return freed


def with_lic(m, path="gb.lic"):
    with open(path) as f:
        env = gp.Env(params=json.load(f))
    return m.copy(env=env)


In [39]:
import numpy as np
import torch
import torch.nn.functional as F

# from temp.solver.utils import fix_var, repair, set_starts, unfix_var, solve_inst

EVIDENCE_FUNCS = {
    "softplus": (lambda y: F.softplus(y)),
    "relu": (lambda y: F.relu(y)),
    "exp": (lambda y: torch.exp(torch.clamp(y, -10, 10))),
}


def get_predictions(logits, binary_mask):
    probs = torch.softmax(logits, axis=1)
    preds = probs[:, 1]
    probs = _to_numpy(probs)
    preds = _to_numpy(preds).squeeze()
    preds[binary_mask] = preds[binary_mask].round()
    return probs, preds


def get_uncertainty(logits, evidence_func_name: str = "softplus"):
    evidence = EVIDENCE_FUNCS[evidence_func_name](logits)
    alpha = evidence + 1
    uncertainty = logits.shape[1] / torch.sum(alpha, dim=1, keepdim=True)
    return uncertainty


def get_threshold(uncertainty: torch.Tensor, r_min: float = 0.4, r_max: float = 0.55):
    q = (r_min + r_max) / 2
    threshold = torch.quantile(uncertainty, q)
    r = (uncertainty <= threshold).float().mean()

    if r > r_max:
        threshold = torch.quantile(uncertainty, r_max)
        return threshold

    if r < r_min:
        threshold = torch.quantile(uncertainty, r_min)
        return threshold

    return threshold


def get_confident_idx(indices, uncertainty, threshold):
    confident_mask = uncertainty <= threshold
    confident_idx = indices[confident_mask]
    return confident_idx.sort()[0]


def solve(inst, prediction, uncertainty, indices, max_iter):
    threshold = get_threshold(uncertainty)
    conf_idxs = get_confident_idx(indices, uncertainty, threshold)
    conf_vals = prediction[conf_idxs]
    bounds = fix_var(inst, conf_idxs, conf_vals)
    print(len(bounds), len(prediction))

    min_q = sum(uncertainty <= threshold) / len(uncertainty)
    max_q = 1.0
    dq = (max_q - min_q) / (max_iter - 1)

    fixed = set(conf_idxs.tolist())
    freed = set(repair(inst, fixed, bounds))
    for i in range(1, max_iter):
        sol = solve_inst(inst)
        q = max_q - dq * i
        threshold = np.quantile(uncertainty, q)
        conf_idxs = get_confident_idx(indices, uncertainty, threshold)
        to_unfix = list(fixed - set(conf_idxs.tolist()))
        to_unfix = [i for i in to_unfix if i not in freed]
        unfix_var(inst, to_unfix, [bounds[i] for i in to_unfix])
        starts = {i: sol[i] for i in to_unfix}
        starts.update({i: sol[i] for i in freed})
        set_starts(inst, starts)
    return sol


def _to_numpy(tensor_obj):
    return tensor_obj.cpu().detach().numpy()


In [ ]:
indices = torch.arange(0, len(unc), dtype=int)
unc = unc.squeeze()
to_solve = m.copy()
to_solve.setParam("TimeLimit", 40)
to_solve.setParam("NoRelHeurTime", 20)
solve(to_solve, preds, unc, indices, 5)

Set parameter TimeLimit to value 40
Set parameter NoRelHeurTime to value 20
243 512
Set parameter IISMethod to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  40
NoRelHeurTime  20
IISMethod  0

IIS computation: initial model status unknown, solving to determine model status
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
IIS runtime: 0.01 seconds (0.00 work units)
Cannot compute IIS on a feasible model
Set parameter IISMethod to value -1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22

Found heuristic solution: objective 241.0000000
Found heuristic solution: objective 221.0000000
Found heuristic solution: objective 208.0000000
Found heuristic solution: objective 189.0000000
Found heuristic solution: objective 183.0000000
Found heuristic solution: objective 179.0000000
Found heuristic solution: objective 177.0000000
Found heuristic solution: objective 175.0000000
Found heuristic solution: objective 148.0000000
Found heuristic solution: objective 143.0000000
Found heuristic solution: objective 141.0000000
Found heuristic solution: objective 136.0000000
Found heuristic solution: objective 134.0000000
Found heuristic solution: objective 129.0000000
Found heuristic solution: objective 128.0000000
Elapsed time for NoRel heuristic: 6s (best bound 76.8308)
Found heuristic solution: objective 127.0000000
Elapsed time for NoRel heuristic: 11s (best bound 76.8308)
Elapsed time for NoRel heuristic: 17s (best bound 76.8308)
NoRel heuristic complete

Root simplex log...

Iteration

[0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0

In [43]:
gb_m = m.copy()
gb_m.setParam("TimeLimit", 150)
gb_m.setParam("NoRelHeurTime", 60)
gb_m.optimize()

Set parameter TimeLimit to value 150
Set parameter NoRelHeurTime to value 60
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  150
NoRelHeurTime  60

Optimize a model with 512 rows, 512 columns and 26296 nonzeros
Model fingerprint: 0xb6f38ccf
Variable types: 0 continuous, 512 integer (512 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 300.0000000
Presolve time: 0.16s
Presolved: 512 rows, 512 columns, 26296 nonzeros
Variable types: 0 continuous, 512 integer (512 binary)
Starting NoRel heuristic
Found heuristic solution: objective 269.0000000
Found heuristic solution: objective 264.0000000
Found heuristic solution: objective 233.00000